# Supervisor (Polyglot)

In [ ]:
#!import ../../lib/fsharp/Notebooks.dib
#!import ../../lib/fsharp/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/6.0.1-preview.1/lib/net6.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"
#r @"../../../../../../../.nuget/packages/argu/6.1.5/lib/netstandard2.0/Argu.dll"
#r @"../../../../../../../.nuget/packages/system.commandline/2.0.0-beta4.22272.1/lib/net6.0/System.CommandLine.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.http.connections.common/7.0.0/lib/net7.0/Microsoft.AspNetCore.Http.Connections.Common.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.http.connections.client/7.0.0/lib/net7.0/Microsoft.AspNetCore.Http.Connections.Client.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.common/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Common.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.client/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Client.dll"
#r @"../../../../../../../.nuget/packages/microsoft.aspnetcore.signalr.client.core/7.0.0/lib/net7.0/Microsoft.AspNetCore.SignalR.Client.Core.dll"
#r @"../../../../../../../.nuget/packages/fsharp.json/0.4.1/lib/netstandard2.0/FSharp.Json.dll"

In [ ]:
#!import ../../lib/fsharp/Common.fs
#!import ../../lib/fsharp/CommonFSharp.fs
#!import ../../lib/fsharp/Threading.fs
#!import ../../lib/fsharp/Async.fs
#!import ../../lib/fsharp/AsyncSeq.fs
#!import ../../lib/fsharp/Networking.fs
#!import ../../lib/fsharp/Runtime.fs
#!import ../../lib/fsharp/FileSystem.fs

In [ ]:
open Common
open FileSystem.Operators
open Microsoft.AspNetCore.SignalR.Client

## sendJson

In [ ]:
let inline sendJson (port : int) (json : string) = async {
    let! portOpen = Networking.testPortOpen port
    if portOpen then
        try
            let connection = HubConnectionBuilder().WithUrl($"http://127.0.0.1:{port}").Build()
            do! connection.StartAsync () |> Async.AwaitTask
            let! result = connection.InvokeAsync<string>("ClientToServerMsg", json) |> Async.AwaitTask
            do! connection.StopAsync () |> Async.AwaitTask
            trace Debug (fun () -> $"sendJson / port: {port} / json: {json} / result.Length: {result |> Option.ofObj |> Option.map String.length}") getLocals
            return Some result
        with ex ->
            trace Critical (fun () -> $"sendJson / port: {port} / json: {json} / ex: {ex |> printException}") getLocals
            return None
    else
        trace Debug (fun () -> "sendJson / error: port not open") getLocals
        return None
}

## sendObj

In [ ]:
let inline sendObj port obj =
    obj
    |> System.Text.Json.JsonSerializer.Serialize
    |> sendJson port

## awaitCompiler

In [ ]:
type VSCPos = {| line : int; character : int |}
type VSCRange = VSCPos * VSCPos
type RString = VSCRange * string
type TracedError = {| trace : string list; message : string |}
type ClientErrorsRes =
    | FatalError of string
    | TracedError of TracedError
    | PackageErrors of {| uri : string; errors : RString list |}
    | TokenizerErrors of {| uri : string; errors : RString list |}
    | ParserErrors of {| uri : string; errors : RString list |}
    | TypeErrors of {| uri : string; errors : RString list |}

In [ ]:
let inline awaitCompiler port cancellationToken = async {
    let! ct =
        cancellationToken
        |> Option.defaultValue System.Threading.CancellationToken.None
        |> Async.mergeCancellationTokenWithDefaultAsync

    let cts = new System.Threading.CancellationTokenSource ()

    let compiler = MailboxProcessor.Start (fun inbox -> async {
        let! availablePort = Networking.getAvailablePort (Some 60) port
        if availablePort <> port then
            inbox.Post port
        else
            let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

            let compilerPath =
                repositoryRoot </> "deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release"
                |> System.IO.Path.GetFullPath

            let dllPath = compilerPath </> "Spiral.dll"

            let! exitCode, result =
                Runtime.executeWithOptionsAsync
                    {
                        Command = $@"dotnet ""{dllPath}"" --port {availablePort} --default-int i32 --default-float f64"
                        CancellationToken = Some ct
                        WorkingDirectory = None
                        OnLine = Some <| fun { Line = line } -> async {
                            if line |> String.contains $"Server bound to: http://localhost:{availablePort}" then
                                do! Networking.waitForPortAccess (Some 500) true availablePort |> Async.Ignore

                                let rec loop retry = async {
                                    let getLocals () = $"port: {availablePort} / retry: {retry} / {getLocals ()}"
                                    try
                                        let pingObj = {| Ping = true |}
                                        let! pingResult = pingObj |> sendObj availablePort
                                        trace Verbose (fun () -> $"awaitCompiler / Ping / result: {pingResult}") getLocals
                                    with ex ->
                                        trace Verbose (fun () -> $"awaitCompiler / Ping / ex: {ex |> printException}") getLocals
                                        do! Async.Sleep 10
                                        do! loop (retry + 1)
                                }
                                do! loop 0
                                inbox.Post availablePort
                        }
                    }
            trace Debug (fun () -> $"awaitCompiler / exitCode: {exitCode} / result: {result}") getLocals
            cts.Cancel ()
    }, ct)

    let! serverPort = compiler.Receive ()

    let connection = HubConnectionBuilder().WithUrl($"http://127.0.0.1:{serverPort}").Build ()
    do! connection.StartAsync () |> Async.AwaitTask

    let event = Event<_> ()
    let disposable = connection.On<string> ("ServerToClientMsg", event.Trigger)
    let stream =
        FSharp.Control.AsyncSeq.unfoldAsync
            (fun () -> async {
                let! msg = event.Publish |> Async.AwaitEvent
                return Some (msg |> FSharp.Json.Json.deserialize<ClientErrorsRes>, ())
            })
            ()

    let disposable =
        newDisposable (fun () ->
            disposable.Dispose ()
            connection.StopAsync () |> Async.AwaitTask |> Async.StartImmediate
        )

    return
        serverPort,
        stream,
        cts.Token,
        disposable
}

## getFileUri

In [ ]:
let inline getFileUri (path : string) =
    let path =
        if Runtime.isWindows () |> not
        then path
        else $"{path.[0] |> System.Char.ToLower}{path.[1..]}" |> String.replace "\\" "/"
    $"file:///{path |> String.trimStart [| '/' |]}"

In [ ]:
let inline getFilePathFromUri uri =
    match System.Uri.TryCreate (uri, System.UriKind.Absolute) with
    | true, uri -> uri.AbsolutePath |> System.IO.Path.GetFullPath
    | _ -> failwith "invalid uri"

In [ ]:
let inline getCompilerPort () =
    13805

## buildFile

In [ ]:
let inline buildFile timeout port cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let fileDir = fullPath |> System.IO.Path.GetDirectoryName
    let fileName = fullPath |> System.IO.Path.GetFileNameWithoutExtension
    let! code = fullPath |> FileSystem.readAllTextAsync

    let eventFilter = function
        | FileSystem.FileSystemChange.Changed (path, _) when path = $"{fileName}.fsx" -> true
        | _ -> false

    let stream, disposable = fileDir |> FileSystem.watchDirectory eventFilter
    use _ = disposable

    let token, disposable = Threading.newDisposableToken cancellationToken
    use _ = disposable

    let! serverPort, errors, ct, disposable = awaitCompiler port (Some token)
    use _ = disposable

    let fsxContentSeq =
        stream
        |> FSharp.Control.AsyncSeq.choose (function
            | _, (FileSystem.FileSystemChange.Changed (path, Some content) as event)
                when event |> eventFilter
                ->
                Some content
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.map (fun content ->
            Some (content |> String.replace "\r\n" "\n"), None
        )

    let inline printErrorData (data : {| uri : string; errors : RString list |}) =
        let fileName = data.uri |> System.IO.Path.GetFileName
        let errors =
            data.errors
            |> List.map snd
            |> String.concat "\n"
        $"{fileName}:\n{errors}"

    let errorsSeq =
        errors
        |> FSharp.Control.AsyncSeq.choose (fun error ->
            match error with
            | FatalError message ->
                Some (message, error)
            | TracedError data ->
                Some (data.message, error)
            | PackageErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | TokenizerErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | ParserErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | TypeErrors data when data.errors |> List.isEmpty |> not ->
                Some (data |> printErrorData, error)
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.map (fun (message, error) ->
            None, Some (message, error)
        )

    let timerSeq =
        1000
        |> FSharp.Control.AsyncSeq.intervalMs
        |> FSharp.Control.AsyncSeq.map (fun _ -> None, None)

    let outputSeq =
        [ fsxContentSeq; errorsSeq; timerSeq ]
        |> FSharp.Control.AsyncSeq.mergeAll

    let! outputChild =
        ((None, [], 0), outputSeq)
        ||> FSharp.Control.AsyncSeq.scan (
            fun (fsxContentResult, errors, typeErrorCount) (fsxContent, error) ->
                match fsxContent, error with
                | Some fsxContent, None -> Some fsxContent, errors, typeErrorCount
                | None, Some (_, FatalError "File main has a type error somewhere in its path.") ->
                    fsxContentResult, errors, typeErrorCount + 1
                | None, Some error -> fsxContentResult, error :: errors, typeErrorCount
                | None, None when typeErrorCount >= 1 ->
                    fsxContentResult, errors, typeErrorCount + 1
                | _ -> fsxContentResult, errors, typeErrorCount
        )
        |> FSharp.Control.AsyncSeq.takeWhileInclusive (fun (fsxContent, errors, typeErrorCount) ->
            trace Debug (fun () -> $"buildFile / takeWhileInclusive / fsxContent: {fsxContent |> Option.defaultValue System.String.Empty |> String.ellipsis 750} / errors: {errors} / typeErrorCount: {typeErrorCount}") getLocals
            match fsxContent, errors with
            | None, [] when typeErrorCount > 2 -> false
            | None, [] -> true
            | _ -> false
        )
        |> FSharp.Control.AsyncSeq.tryLast
        |> Async.withCancellationToken ct
        |> Async.catch
        |> Async.runWithTimeoutAsync timeout
        |> Async.StartChild

    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! _fileOpenResult = fileOpenObj |> sendObj serverPort

    do! Async.Sleep 60

    let buildFileObj = {| BuildFile = {| uri = fullPath |> getFileUri; backend = "Fsharp" |} |}
    let! _buildFileResult = buildFileObj |> sendObj serverPort

    return!
        outputChild
        |> Async.map (function
            | Some (Ok (Some (message, errors, _))) -> message, errors |> List.distinct |> List.rev
            | Some (Error ex) ->
                trace Critical (fun () -> $"buildFile / error: {ex |> printException}") getLocals
                None, []
            | _ -> None, []
        )
}

## persistCode

In [ ]:
let inline persistCode code = async {
    let tempDir = FileSystem.createTempDirectory ()

    let mainPath = tempDir </> "main.spi"
    do! code |> FileSystem.writeAllTextAsync mainPath

    let repositoryRoot = FileSystem.getSourceDirectory () |> FileSystem.findParent ".paket" false

    let spiprojPath = tempDir </> "package.spiproj"
    let spiprojCode =
        $"""packageDir: {repositoryRoot </> "lib"}
packages:
    |core-
    spiral-
modules:
    main
"""
    do! spiprojCode |> FileSystem.writeAllTextAsync spiprojPath

    let disposable = newDisposable (fun () ->
        ()
        // tempDir |> FileSystem.deleteDirectoryAsync |> Async.Ignore |> Async.RunSynchronously
    )

    return mainPath, disposable
}

## buildCode

In [ ]:
let inline buildCode timeout cancellationToken code = async {
    let! mainPath, disposable = persistCode code
    use _ = disposable
    let port = getCompilerPort ()
    return! mainPath |> buildFile timeout port cancellationToken
}

In [ ]:
//// test

let buildCode timeout cancellationToken code = buildCode timeout cancellationToken code

In [ ]:
//// test

"""inl app () =
    console.write_line "text"
    1i32

inl main () =
    app
    |> dyn
    |> ignore
"""
|> buildCode 15000 None
|> Async.runWithTimeout 15000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some """let rec closure0 () () : int32 =
    let v0 : string = "text"
    System.Console.WriteLine v0
    1
let v0 : (unit -> int32) = closure0()
()
""",
        []
    )
)

00:00:00 #1 [Debug] runWithTimeoutAsync / timeout: 60
00:00:00 #2 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-1005> }
00:00:01 #3 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:01 #4 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:01 #5 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:01 #6 [Debug] runWithTimeoutAsync / timeout: 500
00:00:01 #7 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:02 #8 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:02 #9 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 

In [ ]:
//// test

""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> _equal None

00:00:06 #21 [Debug] runWithTimeoutAsync / timeout: 60
00:00:06 #22 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-1005> }
00:00:07 #23 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:07 #24 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:07 #25 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:07 #26 [Debug] runWithTimeoutAsync / timeout: 500
00:00:08 #27 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:08 #28 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:08 #29 [Verbose] > Server bound to

In [ ]:
//// test

"inl app () =
    0i32

inl a = 1

inl main () =
    app
    |> dyn
    |> ignore
"
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "main.spi:
Global inl/let statements should all return functions known at parse time." ]
    )
)

00:00:17 #48 [Debug] runWithTimeoutAsync / timeout: 60
00:00:17 #49 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-1005> }
00:00:17 #50 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:17 #51 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:17 #52 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:18 #54 [Debug] runWithTimeoutAsync / timeout: 500
00:00:18 #56 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:18 #57 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:18 #58 [Verbose] > Server bound to

In [ ]:
//// test

"""inl main () =
    1i32 / 0i32
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "An attempt to divide by zero has been detected at compile time." ]
    )
)

00:00:19 #67 [Debug] runWithTimeoutAsync / timeout: 60
00:00:19 #68 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-1005> }
00:00:19 #69 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:19 #70 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:19 #71 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:20 #72 [Debug] runWithTimeoutAsync / timeout: 500
00:00:20 #74 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:20 #75 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:20 #76 [Verbose] > Server bound to

In [ ]:
//// test

"""inl main () =
    1 + ""
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [
            "main.spi:
Constraint satisfaction error.
Got: string
Fails to satisfy: number"
        ]
    )
)

00:00:23 #89 [Debug] runWithTimeoutAsync / timeout: 60
00:00:23 #90 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-1005> }
00:00:23 #91 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:23 #92 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:24 #93 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:24 #95 [Debug] runWithTimeoutAsync / timeout: 500
00:00:24 #96 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:24 #97 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:24 #98 [Verbose] > Server bound to

In [ ]:
//// test

"""inl main () =
    x + y
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [
            "main.spi:
Unbound variable: x.
Unbound variable: y."
        ]
    )
)

00:00:27 #112 [Debug] runWithTimeoutAsync / timeout: 60
00:00:27 #113 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-1005> }
00:00:27 #114 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:27 #115 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:28 #116 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:28 #118 [Debug] runWithTimeoutAsync / timeout: 500
00:00:28 #119 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:28 #120 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:28 #121 [Verbose] > Server

In [ ]:
//// test

"""union a =
    | B
    | c

inl main () =
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "main.spi:
Expected: uppercase variable" ]
    )
)

00:00:31 #133 [Debug] runWithTimeoutAsync / timeout: 60
00:00:31 #134 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-1005> }
00:00:31 #136 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:31 #137 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:31 #138 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:32 #139 [Debug] runWithTimeoutAsync / timeout: 500
00:00:32 #140 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:32 #141 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:32 #142 [Verbose] > Server

In [ ]:
//// test

"""
/// abc
inl main () =
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "main.spi:
Expected: whitespace" ]
    )
)

00:00:33 #152 [Debug] runWithTimeoutAsync / timeout: 60
00:00:33 #153 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-1005> }
00:00:34 #154 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:34 #155 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:34 #157 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:34 #158 [Debug] runWithTimeoutAsync / timeout: 500
00:00:35 #159 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:35 #160 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:35 #161 [Verbose] > Server

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox ()
    ()
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "Cannot apply a forall with a term." ]
    )
)

00:00:35 #169 [Debug] runWithTimeoutAsync / timeout: 60
00:00:35 #170 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-1005> }
00:00:36 #171 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:36 #172 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:36 #174 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:36 #175 [Debug] runWithTimeoutAsync / timeout: 500
00:00:37 #176 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:37 #177 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:37 #178 [Verbose] > Server

In [ ]:
//// test

"""
inl main () =
    real
        inl real_unbox forall a. (obj : a) : a =
            typecase obj with
            | _ => obj
        real_unbox `i32 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        None,
        [ "The main function should not have a forall." ]
    )
)

00:00:40 #192 [Debug] runWithTimeoutAsync / timeout: 60
00:00:40 #193 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-1005> }
00:00:41 #194 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:41 #195 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:41 #197 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:41 #198 [Debug] runWithTimeoutAsync / timeout: 500
00:00:42 #199 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:42 #200 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:42 #201 [Verbose] > Server

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.1 (fun x => x ** 2) 0 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
|> _equal (
    Some (
        Some "0.3325000000000001\n",
        []
    )
)

00:00:46 #216 [Debug] runWithTimeoutAsync / timeout: 60
00:00:46 #217 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-1005> }
00:00:47 #219 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:47 #220 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:47 #221 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:47 #222 [Debug] runWithTimeoutAsync / timeout: 500
00:00:47 #223 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:47 #225 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:47 #226 [Verbose] awaitCompiler / Ping / result: S

In [ ]:
//// test

"""
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start) : list f64

type integration = (f64 -> f64) -> f64 -> f64 -> f64

inl integral dt : integration =
    fun f a b =>
        init_series (a + dt / 2) (b - dt / 2) dt
        |> listm.map (f >> (*) dt)
        |> listm.fold (+) 0

inl main () =
    integral 0.01 (fun x => x ** 2) 0 1
"""
|> buildCode 10000 None
|> Async.runWithTimeout 10000
|> Option.map (fun (fsxContent, errors) -> fsxContent, errors |> List.map fst)
// |> _equal None
// |> fun x -> printfn $"{x.ToDisplayString ()}"

Some((Some(0.33332500000000004\n), [])) Value (Some(0.33332500000000004\n), []) Item1 Some(0.33332500000000004\n) Value 0.33332500000000004
 Item2 [ ]

00:00:51 #240 [Debug] runWithTimeoutAsync / timeout: 60
00:00:51 #241 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:buildCode@3-1005> }
00:00:52 #243 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:00:52 #244 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:52 #245 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:52 #246 [Debug] runWithTimeoutAsync / timeout: 500
00:00:52 #248 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:52 #249 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:00:52 #250 [Verbose] > Server

## getFileTokenRange

In [ ]:
let inline getFileTokenRange port cancellationToken path = async {
    let fullPath = path |> System.IO.Path.GetFullPath
    let! code = fullPath |> FileSystem.readAllTextAsync
    let lines = code |> String.split [| '\n' |]

    let token, disposable = Threading.newDisposableToken cancellationToken
    use _ = disposable

    let! serverPort, _errors, ct, disposable = awaitCompiler port (Some token)
    use _ = disposable

    let fileOpenObj = {| FileOpen = {| uri = fullPath |> getFileUri; spiText = code |} |}
    let! _fileOpenResult = fileOpenObj |> sendObj serverPort

    let fileTokenRangeObj =
        {|
            FileTokenRange =
                {|
                    uri = fullPath |> getFileUri
                    range =
                        [|
                            {| line = 0; character = 0 |}
                            {| line = lines.Length - 1; character = lines.[lines.Length - 1].Length |}
                        |]
                |}
        |}
    let! fileTokenRangeResult =
        fileTokenRangeObj
        |> sendObj serverPort
        |> Async.withCancellationToken ct

    return fileTokenRangeResult |> Option.map FSharp.Json.Json.deserialize<int array>
}

## getCodeTokenRange

In [ ]:
let inline getCodeTokenRange cancellationToken code = async {
    let! mainPath, disposable = persistCode code
    use _ = disposable
    let port = getCompilerPort ()
    return! mainPath |> getFileTokenRange port cancellationToken
}

In [ ]:
//// test

"""inl main () = ()"""
|> getCodeTokenRange None
|> Async.runWithTimeout 10000
|> Option.flatten
|> _equal (Some [| 0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 8; 0; 0; 1; 1; 8; 0 |])

00:01:12 #277 [Debug] runWithTimeoutAsync / timeout: 60
00:01:12 #278 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:it@4-434> }
00:01:12 #280 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:01:12 #281 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:13 #282 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:01:13 #283 [Debug] runWithTimeoutAsync / timeout: 500
00:01:13 #284 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:01:13 #285 [Verbose] awaitCompiler / Ping / result: Some(null) / port: 13805 / retry: 0
00:01:13 #286 [Verbose] > Server bound t

In [ ]:
//// test

"""inl main () = 1i32"""
|> getCodeTokenRange None
|> Async.runWithTimeout 10000
|> Option.flatten
|> _equal (Some [| 0; 0; 3; 7; 0; 0; 4; 4; 0; 0; 0; 5; 1; 8; 0; 0; 1; 1; 8; 0; 0; 2; 1; 4; 0; 0;
2; 1; 3; 0; 0; 1; 3; 12; 0 |])

00:01:19 #295 [Debug] runWithTimeoutAsync / timeout: 60
00:01:19 #296 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:it@4-801> }
00:01:20 #298 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral
00:01:20 #299 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:01:20 #300 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:01:20 #301 [Debug] runWithTimeoutAsync / timeout: 500
00:01:20 #302 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:01:20 #303 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:01:20 #304 [Verbose] awaitCompiler / Ping / result: Some(null

## Arguments

In [ ]:
[<RequireQualifiedAccess>]
type Arguments =
    | Build_File of string * string
    | File_Token_Range of string * string
    | Execute_Command of string
    | Timeout of int
    | Port of int

    interface Argu.IArgParserTemplate with
        member s.Usage =
            match s with
            | Build_File _ -> nameof Build_File
            | File_Token_Range _ -> nameof File_Token_Range
            | Execute_Command _ -> nameof Execute_Command
            | Timeout _ -> nameof Timeout
            | Port _ -> nameof Port

In [ ]:
//// test

Argu.ArgumentParser.Create<Arguments>().PrintUsage ()

USAGE: dotnet.exe [--help] [--build-file <string> <string>] [--file-token-range <string> <string>] [--execute-command <string>] [--timeout <int>] [--port <int>]

OPTIONS:

    --build-file <string> <string>
                          Build_File
    --file-token-range <string> <string>
                          File_Token_Range
    --execute-command <string>
                          Execute_Command
    --timeout <int>       Timeout
    --port <int>          Port
    --help                display this list of options.


## main

In [ ]:
let main args =
    let argsMap = args |> Runtime.parseArgsMap<Arguments>

    let buildFileActions =
        argsMap
        |> Map.tryFind (nameof Arguments.Build_File)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.Build_File (inputPath, outputPath) -> Some (inputPath, outputPath)
            | _ -> None
        )

    let fileTokenRangeActions =
        argsMap
        |> Map.tryFind (nameof Arguments.File_Token_Range)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.File_Token_Range (inputPath, outputPath) -> Some (inputPath, outputPath)
            | _ -> None
        )

    let executeCommandActions =
        argsMap
        |> Map.tryFind (nameof Arguments.Execute_Command)
        |> Option.defaultValue []
        |> List.choose (function
            | Arguments.Execute_Command command -> Some command
            | _ -> None
        )

    let timeout =
        match argsMap |> Map.tryFind (nameof Arguments.Timeout) with
        | Some [ Arguments.Timeout timeout ] -> timeout
        | _ -> 60000 * 60

    let port =
        match argsMap |> Map.tryFind (nameof Arguments.Port) with
        | Some [ Arguments.Port port ] -> Some port
        | _ -> None

    async {
        let! buildFileResult =
            buildFileActions
            |> List.map (fun (inputPath, outputPath) -> async {
                let port = port |> Option.defaultWith getCompilerPort
                let! outputCode, errors = inputPath |> buildFile timeout port None

                errors
                |> List.map snd
                |> List.iter (fun error ->
                    trace Critical (fun () -> $"main / error: {error}") getLocals
                )

                match outputCode with
                | Some outputCode ->
                    do! outputCode |> FileSystem.writeAllTextAsync outputPath
                    return 0
                | None ->
                    return 1
            })
            |> Async.Sequential

        let! fileTokenRangeResult =
            fileTokenRangeActions
            |> List.map (fun (inputPath, outputPath) -> async {
                let port = port |> Option.defaultWith getCompilerPort
                let! tokenRange = inputPath |> getFileTokenRange port None
                match tokenRange with
                | Some tokenRange ->
                    do! tokenRange |> FSharp.Json.Json.serialize |> FileSystem.writeAllTextAsync outputPath
                    return 0
                | None ->
                    return 1
            })
            |> Async.Sequential

        let! executeCommandResult =
            executeCommandActions
            |> List.map (fun command -> async {
                let port = port |> Option.defaultWith getCompilerPort

                let localToken, disposable = Threading.newDisposableToken None
                use _ = disposable

                let! serverPort, _errors, compilerToken, disposable = awaitCompiler port (Some localToken)
                use _ = disposable

                let! exitCode, result =
                    Runtime.executeWithOptionsAsync
                        {
                            Command = command
                            CancellationToken = Some compilerToken
                            WorkingDirectory = None
                            OnLine = None
                        }

                trace Debug (fun () -> $"main / executeCommand / exitCode: {exitCode}") getLocals

                return exitCode
            })
            |> Async.Sequential

        return
            [| buildFileResult; fileTokenRangeResult; executeCommandResult |]
            |> Array.collect id
            |> Array.sum
    }
    |> Async.runWithTimeout timeout
    |> Option.defaultValue 1

In [ ]:
//// test

let args =
    System.Environment.GetEnvironmentVariable "ARGS"
    |> Runtime.splitArgs
    |> Seq.toArray

match args with
| [||] -> 0
| args -> if main args = 0 then 0 else failwith "main failed"

0